# SQL Analytics
Builds the SQLite tables used for analytics and shows sample queries.
Ensure `gtd_cleaned.csv` exists in `data/` (created by `data_preprocessing.ipynb`).

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Adjust if needed
base_path = '/content/drive/MyDrive/LABS/Project-1-GTD-Analytics'
# Load cleaned data
df_clean = pd.read_csv(f'{base_path}/data/gtd_cleaned.csv')
print('Loaded cleaned data:', df_clean.shape)

In [ ]:
# CREATE SQLITE DATABASE - derived from cleaned data
engine = create_engine(f'sqlite:///{base_path}/data/gtd_analytics.db')

# 1. MONTHLY TRENDS TABLE
print("Creating monthly trends table...")
monthly = df_clean.groupby('year_month').agg({
    'eventid': 'count',
    'casualties': 'sum',
    'nkill': 'sum',
    'success': 'mean'
}).round(2).reset_index()
monthly['year_month'] = monthly['year_month'].astype(str)
monthly.columns = ['year_month', 'attacks', 'total_casualties', 'fatalities', 'success_rate']
monthly.to_sql('monthly_trends', engine, if_exists='replace', index=False)

# 2. REGIONAL SUMMARY TABLE
print("Creating regional summary...")
region = df_clean.groupby('region_txt').agg({
    'eventid': 'count',
    'casualties': 'sum',
    'country_txt': 'nunique'
}).round(2).reset_index()
region.columns = ['region', 'attacks', 'casualties', 'countries']
region.to_sql('regional_summary', engine, if_exists='replace', index=False)

# 3. MAIN ATTACKS TABLE (SQLite-safe version)
print("Creating main attacks table...")
df_sqlite = df_clean[['eventid', 'iyear', 'imonth', 'iday', 'country_txt', 'region_txt',
                     'city', 'latitude', 'longitude', 'attacktype1_txt', 'nkill',
                     'nwound', 'success', 'year_month', 'casualties', 'severity']].copy()
df_sqlite['year_month'] = df_sqlite['year_month'].astype(str)
df_sqlite.to_sql('gtd_attacks', engine, if_exists='replace', index=False)

# Verify
print('
SUCCESS! Tables created:')
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", engine))

print('
Monthly trends sample:')
print(pd.read_sql("SELECT * FROM monthly_trends LIMIT 5;", engine))

In [ ]:
# Example queries for exploration
print('Top regions by attacks:')
print(pd.read_sql("SELECT region, attacks FROM regional_summary ORDER BY attacks DESC LIMIT 10;", engine))

print('Attacks in a given month (sample):')
print(pd.read_sql("SELECT * FROM monthly_trends WHERE year_month LIKE '2015-%' LIMIT 5;", engine))